# 🔧 Fine-Tune TinyLlama on Colab (No HF Login, No LoRA, No Unsloth)

In [ ]:

!pip install transformers datasets accelerate


In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=False)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)


In [ ]:

from datasets import Dataset

data = [
    {"text": "### Question: What is 2 + 2?\n### Answer: 4"},
    {"text": "### Question: What is 3 + 5?\n### Answer: 8"},
]

dataset = Dataset.from_list(data)


In [ ]:

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


In [ ]:

from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=1,
    num_train_epochs=2,
    fp16=True,
    logging_steps=1,
    save_steps=10,
    save_total_limit=1,
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()


In [ ]:

model.save_pretrained("./model")
tokenizer.save_pretrained("./model")


In [ ]:

from transformers import pipeline

pipe = pipeline("text-generation", model="./model", tokenizer=tokenizer)
pipe("### Question: What is 10 + 5?\n### Answer:", max_new_tokens=50)
